# E. coli Data Processing

This notebook creates the datasets needed to train the E. coli model. The following datasets are created:

#### E. coli Language Model 
This dataframe will contain the E. coli genome as long text strings, parsed to remove poorly sequenced regions (N heavy)

#### E. coli Promoter Classification
This dataframe will contain E. coli promoters, defined as -150/+50 base pairs from each annotated transcription start site. The classification data will also include negative examples of the same length taken from regions not including a TSS. Taking promoter sequences based on TSS locations runs the risk of putting highly similar promoters into the dataset. To construct meaningful test and validation sets to evaluate the model, we must ensure that sequences in the test/validation sets are sufficiently different from sequences in the training set. This is done using a method described below.

#### Data Source
E. coli genomic data and TSS locations are taken from the E. coli reference genome genbank file available at [NCBI](https://www.ncbi.nlm.nih.gov/genome/?term=escherichia%20coli)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.text import *
from Bio import Seq
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import FeatureLocation, CompoundLocation
import networkx as nx

In [3]:
sys.path.append("../..")
from utils import *

In [4]:
path = Path('F:/genome/e_coli/')

# Language Model Data

In [5]:
fname = 'GCF_000005845.2_ASM584v2_genomic.fna'

In [6]:
data = process_fasta(path/fname, 2000, 900)

In [7]:
val_pct = 0.15
cut = int(len(data)*val_pct) + 1

train_df = pd.DataFrame(data[:cut], columns=['Sequence'])
valid_df = pd.DataFrame(data[cut:], columns=['Sequence'])
train_df['is_train'] = 1
valid_df['is_train'] = 0

data_df = pd.concat([train_df, valid_df])
data_df.to_csv(path/'e_coli_lm_data.csv', index=False)

In [8]:
data_df.head()

,Sequence,is_train
0,AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATT...,1
1,GGTTTCACCGCCGGTAATGAAAAAGGCGAACTGGTGGTGCTTGGAC...,1
2,AGCTGGCTGAAGAATAAACATATCGACTTACGTGTCTGCGGTGTTG...,1
3,CCGTTGGTACTGCGCGGATATGGTGCGGGCAATGACGTTACAGCTG...,1
4,CGCTCTGTGTGACAAGCCGGAAACCGCCCAGCGCGTTGCCGACTGG...,1


# Classification Data

In [5]:
fname = 'GCF_000005845.2_ASM584v2_genomic.gbff'

In [11]:
GB = next(SeqIO.parse(path/fname, "genbank"))

In [12]:
# Extract promoter regions
# All sequences saved as sense strands
def extract_promoter(feature):
    quals = list(feature.qualifiers.keys())
    
    if 'gene' in quals:
        gene = feature.qualifiers['gene']
    else:
        gene = None
        
    if 'locus_tag' in quals:
        locus = feature.qualifiers['locus_tag']
    else:
        locus = None
        
    cds_loc = str(feature.location)
    cds_start = feature.location.start.position
    cds_end = feature.location.end.position
    p_start = cds_start - 100
    p_end = cds_end + 100
    
    if feature.strand == -1:
        orient = 'reverse'
        promoter = GB[cds_end-50:p_end].reverse_complement().seq.__str__()
        promoter_loc = f"[{cds_end-50}:{p_end}]" + f"{cds_loc[-3:]}"
                
    else:
        orient = 'forward'
        promoter = GB[p_start:cds_start+50].seq.__str__()
        promoter_loc = f"[{p_start}:{cds_start+50}]" + f"{cds_loc[-3:]}"
                    
    promoter_data = [gene, locus, cds_loc, promoter_loc, orient, promoter, 1]
    
    return promoter_data

In [13]:
# Extract negative examples
def extract_gene(feature):
    quals = list(feature.qualifiers.keys())
    
    if 'gene' in quals:
        gene = feature.qualifiers['gene']
    else:
        gene = None
        
    if 'locus_tag' in quals:
        locus = feature.qualifiers['locus_tag']
    else:
        locus = None
        
    cds_loc = str(feature.location)
    cds_start = feature.location.start.position
    cds_end = feature.location.end.position
    
    gene_len = (cds_end-50) - (cds_start + 50)
    
    if gene_len > 150:
        rand_start = np.random.randint(cds_start+50, cds_end-200)
        rand_gene = GB[rand_start:rand_start+150]
        rand_gene_loc = f"[{rand_start}:{rand_start+150}]" + f"{cds_loc[-3:]}"
            
        if feature.strand == -1:
            orient = 'reverse'
            rand_gene = rand_gene.reverse_complement().seq.__str__()
            
        else:
            orient = 'forward'
            rand_gene = rand_gene.seq.__str__()
            
        gene_data = [gene, locus, cds_loc, rand_gene_loc, orient, rand_gene, 0]
        return gene_data

    else:
        return False

In [10]:
proms = []
for feature in GB.features:
    if feature.type == 'CDS':
        proms.append(extract_promoter(feature))

In [11]:
prom_df = pd.DataFrame(proms, columns=['Gene', 'Locus', 'Location', 'Sample Location', 'Orientation', 'Sequence', 'Promoter'])

In [13]:
genes = []
for feature in GB.features:
    if feature.type == 'CDS':
        gene = extract_gene(feature)
        if gene:
            genes.append(gene)

In [14]:
gene_df = pd.DataFrame(genes, columns=['Gene', 'Locus', 'Location', 'Sample Location', 'Orientation', 'Sequence', 'Promoter'])

In [16]:
# Approximately 4000 positive and negative examples
prom_df.shape, gene_df.shape

((4357, 7), (4014, 7))

In [17]:
promoter_data = pd.concat([prom_df, gene_df])

In [18]:
promoter_data.shape

(8371, 7)

In [12]:
promoter_data.head()

,Gene,Locus,Location,Sample Location,Orientation,Sequence,Promoter
0,[thrL],[b0001],[189:255](+),[89:239](+),forward,CGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTA...,1
1,[thrA],[b0002],[336:2799](+),[236:386](+),forward,AGGTAACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCC...,1
2,[thrB],[b0003],[2800:3733](+),[2700:2850](+),forward,CCGTTGGTACTGCGCGGATATGGTGCGGGCAATGACGTTACAGCTG...,1
3,[thrC],[b0004],[3733:5020](+),[3633:3783](+),forward,CGTTGCCGACTGGTTGGGTAAGAACTACCTGCAAAATCAGGAAGGT...,1
4,[yaaX],[b0005],[5233:5530](+),[5133:5283](+),forward,GTAACTTAGAGATTAGGATTGCGGAGAATAACAACCGCCGTTCTCA...,1


In [19]:
promoter_data.to_csv(path/'e_coli_promoters.csv', index=False)

# Sequence Similarity

It's highly likely that there are promoter sequences in the dataset that share sequence similarity. We need to make sure similar sequences don't get split into training and validation/test sets.

We use a BLAST search to identify homologies between sequences. We use the information from the BLAST search to create a graph, where each sequence is a node connected to other sequences that share homology. We then grab a maximal independent set from the graph. We can safely put sequences identified as independent into validation/test sets.

Sequences sharing more than 15 base pairs of homology are considered similar.

10% of sequences will be set aside for the test set. Of the 90% of sequences that remain, 10% of those will be set aside for a validation set. The remaining sequences will be used for training.

In [5]:
df = pd.read_csv(path/'e_coli_promoters.csv')

In [6]:
df.head()

,Gene,Locus,Location,Sample Location,Orientation,Sequence,Promoter
0,['thrL'],['b0001'],[189:255](+),[89:239](+),forward,CGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTA...,1
1,['thrA'],['b0002'],[336:2799](+),[236:386](+),forward,AGGTAACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCC...,1
2,['thrB'],['b0003'],[2800:3733](+),[2700:2850](+),forward,CCGTTGGTACTGCGCGGATATGGTGCGGGCAATGACGTTACAGCTG...,1
3,['thrC'],['b0004'],[3733:5020](+),[3633:3783](+),forward,CGTTGCCGACTGGTTGGGTAAGAACTACCTGCAAAATCAGGAAGGT...,1
4,['yaaX'],['b0005'],[5233:5530](+),[5133:5283](+),forward,GTAACTTAGAGATTAGGATTGCGGAGAATAACAACCGCCGTTCTCA...,1


In [7]:
# Write sequences to fasta file
with open('sequences.fasta', 'w') as out:
    for i in range(len(df)):
        out.write('>' + str(df.index[i]) + '\n' + df.Sequence.iloc[i] + '\n')

In [ ]:
!makeblastdb -in sequences.fasta -dbtype 'nucl' -out hom_arm_db

In [ ]:
!blastn -db hom_arm_db -query sequences.fasta -word_size 15 -out out_1.csv -outfmt 6

In [8]:
blast_cols = ['Query', 'Subject', '%ident', 'Length', 'Mismatch', 'Gaps', 'qstart', 'qend',
                 'sstart', 'send', 'evalue', 'bitscore']
df_out = pd.read_csv('out_1.csv', sep='\t', header=None)
df_out.columns = blast_cols
df_out = df_out[df_out.Query != df_out.Subject]
df_out.reset_index(inplace=True, drop=True)

In [9]:
df_inter = df_out[['Query', 'Subject']]
df_inter = df_inter.drop_duplicates()

In [10]:
def link_list(hom, overlaps, hom_names):
    links = overlaps[overlaps.Query == hom].Subject.values
    links = links[np.isin(links, hom_names)]
    links = [(hom, i) for i in links]

    return links

In [11]:
def max_pooling(hom_names_list, hom_overlap_list, link_dict_in):
    G = nx.Graph()
    G.add_nodes_from(hom_names_list)

    for i in hom_overlap_list:
        G.add_edges_from(link_dict_in[i])

    return nx.maximal_independent_set(G)

In [12]:
link_dict = {}
for hom in df_inter.Query.unique():
    link_dict[hom] = link_list(hom, df_inter, df.index.values)

In [13]:
ind_set = np.array(max_pooling(df.index.values, df_inter.Query.unique(), link_dict))

7907 sequences are independent, as defined as having less than 15 base pairs of homology to another sequence in the dataset

In [14]:
len(ind_set)

7907

In [15]:
df['Independent'] = df.index.map(lambda x: x in ind_set)

In [16]:
df.head(10)

,Gene,Locus,Location,Sample Location,Orientation,Sequence,Promoter,Independent
0,['thrL'],['b0001'],[189:255](+),[89:239](+),forward,CGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTA...,1,True
1,['thrA'],['b0002'],[336:2799](+),[236:386](+),forward,AGGTAACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCC...,1,True
2,['thrB'],['b0003'],[2800:3733](+),[2700:2850](+),forward,CCGTTGGTACTGCGCGGATATGGTGCGGGCAATGACGTTACAGCTG...,1,True
3,['thrC'],['b0004'],[3733:5020](+),[3633:3783](+),forward,CGTTGCCGACTGGTTGGGTAAGAACTACCTGCAAAATCAGGAAGGT...,1,True
4,['yaaX'],['b0005'],[5233:5530](+),[5133:5283](+),forward,GTAACTTAGAGATTAGGATTGCGGAGAATAACAACCGCCGTTCTCA...,1,True
5,['yaaA'],['b0006'],[5682:6459](-),[6409:6559](-),reverse,GGACGCGTGGGATGATGTTTCGCAGGAGTAATCACAACTATCGATC...,1,True
6,['yaaJ'],['b0007'],[6528:7959](-),[7909:8059](-),reverse,ACGCAGAAGTTATCAAGTACCTCGTAGCGTATATACTTCTTAAACA...,1,True
7,['talB'],['b0008'],[8237:9191](+),[8137:8287](+),forward,ATAACCGTCTTGTCGGCGGTTGCGCTGACGTTGCGTCGTGATATCA...,1,True
8,['mog'],['b0009'],[9305:9893](+),[9205:9355](+),forward,ACCGGGAAGTCGGTCACGCTACCTCTTCTGAAGCCTGTCTGTCACT...,1,True
9,['satP'],['b0010'],[9927:10494](-),[10444:10594](-),reverse,CACTGCCCCACTGGCTGATTATTATGCCGCGCCCTGAAAACACTAC...,1,True


# Train, Validation, and Test Sets

The test set will contain 830 sequences, 415 promoters and 415 non-promoters.

The validation set will contain 750 sequences, 375 promoters and 375 non-promoters.

In [18]:
test_proms = df[(df.Promoter == 1) & (df.Independent == True)][:415]
val_proms = df[(df.Promoter == 1) & (df.Independent == True)][415:415+375]

test_genes = df[(df.Promoter == 0) & (df.Independent == True)][:415]
val_genes = df[(df.Promoter == 0) & (df.Independent == True)][415:415+375]

In [19]:
val_set = pd.concat([val_genes, val_proms])
val_set['set'] = 'valid'

test_set = pd.concat([test_genes, test_proms])
test_set['set'] = 'test'

In [20]:
test_val = pd.concat([val_set, test_set])

In [21]:
train_data = df[~df.index.isin(test_val.index)]
train_data['set'] = 'train'

C:\Users\GATEWAY\Anaconda2\envs\fastai_v1\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
full_data = pd.concat([train_data, test_val])

In [23]:
full_data.shape

(8371, 9)

In [24]:
full_data.to_csv(path/'e_coli_promoters_dataset.csv', index=False)